## This project Mainly done to classify the images of Raods having cracks and No Cracks

### 40000 images are trained  , where 20,000 images are with road crack and 20,000 images with no_crack

## Convolution Neural Networks with convo , max pooling  , two layers are added and then it is flattened


### The below code cell imports the base libraries called Tensor Flow, Image Data generator


In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

local_zip = '/content/drive/My Drive/Road Crack Detection.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/My Drive/')
zip_ref.close()



Checking for images present in the respective folder

In [ ]:
print(len(os.listdir('/content/drive/My Drive/train/Positive')))
print(len(os.listdir('/content/drive/My Drive/train/Negative')))

20000
20000


Making directories to split the Training and validation sets

In [ ]:
#using os.mkdir creating directories for train and test data

to_create = [
    '/content/drive/My Drive/train/positive-v-negative',
    '/content/drive/My Drive/train/positive-v-negative/training',
    '/content/drive/My Drive/train/positive-v-negative/testing',
    '/content/drive/My Drive/train/positive-v-negative/training/positive',
    '/content/drive/My Drive/train/positive-v-negative/training/negative',
    '/content/drive/My Drive/train/positive-v-negative/testing/positive',
    '/content/drive/My Drive/train/positive-v-negative/testing/negative'
]

for directory in to_create:
    try:
        os.mkdir(directory)
        print(directory, 'created')
    except:
        print(directory, 'failed')

/content/drive/My Drive/train/positive-v-negative created
/content/drive/My Drive/train/positive-v-negative/training created
/content/drive/My Drive/train/positive-v-negative/testing created
/content/drive/My Drive/train/positive-v-negative/training/positive created
/content/drive/My Drive/train/positive-v-negative/training/negative created
/content/drive/My Drive/train/positive-v-negative/testing/positive created
/content/drive/My Drive/train/positive-v-negative/testing/negative created


## Making splits with certain split Ratio 
## It is a UDF function written for general purpose code

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    all_files = []
    
    for file_name in os.listdir(SOURCE):
        file_path = SOURCE + file_name

        if os.path.getsize(file_path):
            all_files.append(file_name)
        else:
            print('{} is zero length, so ignoring'.format(file_name))
    
    n_files = len(all_files)
    split_point = int(n_files * SPLIT_SIZE)
    
    shuffled = random.sample(all_files, n_files)
    
    train_set = shuffled[:split_point]
    test_set = shuffled[split_point:]
    
    for file_name in train_set:
        copyfile(SOURCE + file_name, TRAINING + file_name)
        
    for file_name in test_set:
        copyfile(SOURCE + file_name, TESTING + file_name)


POSITIVE_SOURCE_DIR = "/content/drive/My Drive/train/Positive/"
TRAINING_POSITIVE_DIR = "/content/drive/My Drive/train/positive-v-negative/training/positive/"
TESTING_POSITIVE_DIR = "/content/drive/My Drive/train/positive-v-negative/testing/positive/"
NEGATIVE_SOURCE_DIR = "/content/drive/My Drive/train/Negative/"
TRAINING_NEGATIVE_DIR = "/content/drive/My Drive/train/positive-v-negative/training/negative/"
TESTING_NEGATIVE_DIR = "/content/drive/My Drive/train/positive-v-negative/testing/negative/"

split_size = 0.8
split_data(POSITIVE_SOURCE_DIR, TRAINING_POSITIVE_DIR, TESTING_POSITIVE_DIR, split_size)
split_data(NEGATIVE_SOURCE_DIR, TRAINING_NEGATIVE_DIR, TESTING_NEGATIVE_DIR, split_size)

## Checking for size of images splitted for train and test sets

In [ ]:
print(len(os.listdir('/content/drive/My Drive/train/positive-v-negative/training/positive/')))
print(len(os.listdir('/content/drive/My Drive/train/positive-v-negative/training/negative/')))
print(len(os.listdir('/content/drive/My Drive/train/positive-v-negative/testing/positive/')))
print(len(os.listdir('/content/drive/My Drive/train/positive-v-negative/testing/negative/')))

16000
16000
4000
4000


### Call backs is used to stop the model training when the model is trained to certain limit

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
          def on_epoch_end(self, epoch, logs={}):
                if(logs.get('acc')>0.99):
                    print("\nReached 99% accuracy so cancelling training!")
                    self.model.stop_training = True

## CNN architecture used
## Adding Convolution , Maxpooling Layers


In [ ]:
callbacks=myCallback()
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (2,2), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

## Resizing the images to same size that is 150x150 pixels

In [ ]:
TRAINING_DIR = '/content/drive/My Drive/train/positive-v-negative/training/'
train_datagen = ImageDataGenerator(rescale=1 / 255)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size=64,
    class_mode='binary',
    target_size=(150, 150)
)

VALIDATION_DIR = '/content/drive/My Drive/train/positive-v-negative/testing'
validation_datagen = ImageDataGenerator(rescale= 1/255)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size=64,
    class_mode='binary',
    target_size=(150, 150)
)


Found 32000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.


## Training the model with twenty epochs, and keeping accuracy limit to 99%

In [ ]:
history = model.fit_generator(train_generator,epochs=20,validation_data=validation_generator,callbacks=[callbacks])


Epoch 1/20
500/500 [==============================] - 132s 264ms/step - loss: 0.3892 - acc: 0.9233 - val_loss: 0.0518 - val_acc: 0.9806
Epoch 2/20
500/500 [==============================] - 133s 266ms/step - loss: 0.0921 - acc: 0.9802 - val_loss: 0.0305 - val_acc: 0.9915
Epoch 3/20
500/500 [==============================] - 132s 264ms/step - loss: 0.7821 - acc: 0.9819 - val_loss: 0.1334 - val_acc: 0.9643
Epoch 4/20
500/500 [==============================] - 132s 265ms/step - loss: 0.0649 - acc: 0.9874 - val_loss: 0.0503 - val_acc: 0.9909
Epoch 5/20
500/500 [==============================] - 131s 263ms/step - loss: 0.0455 - acc: 0.9899 - val_loss: 0.0262 - val_acc: 0.9942
Epoch 6/20
500/500 [==============================] - 133s 265ms/step - loss: 0.0385 - acc: 0.9918 - val_loss: 0.0455 - val_acc: 0.9908
Epoch 7/20
 46/500 [=>............................] - ETA: 1:38 - loss: 0.0196 - acc: 0.9949

KeyboardInterrupt: ignored

## Checking with the google images , whether ower model is working fine

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  path = '/content/' + fn
  img = image.load_img(path, target_size=((150, 150)))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + "\n Road has a crack")
  else:
    print(fn + "\n Road has no crack")

Saving 1.jfif to 1.jfif
[1.]
1.jfif
 Road has a crack
